## INITIALIZE DATA FOR PREDICTING FLIGHT DELAYS

# Prepare the DataSet for our pipeline

First we get a handle to our workspace

In [2]:
%pip install azure-ai-ml
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="",  # this will look like xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
    resource_group_name="rg-sbx-aiops",
    workspace_name="romain-mlbox",
)

Note: you may need to restart the kernel to use updated packages.


We download in local a public blob storage account that contains all the data we need, then we create a Dataset that contains every files.

In [3]:
from azure.ai.ml.entities import Data
import os
from azure.storage.blob import BlobServiceClient
from azure.ai.ml.constants import AssetTypes
import shutil

dataset_dir = "./Datasets"

os.makedirs(dataset_dir, exist_ok=True)

account_url = "https://publicdataflights.blob.core.windows.net/"

blob_service_client = BlobServiceClient(account_url=account_url)


# download all blob into local files
container_name = 'dataflights'
container_client = blob_service_client.get_container_client(container_name)
blob_list = container_client.list_blobs()
for blob in blob_list:
    print(blob.name)
    blob_client = blob_service_client.get_blob_client(container_name, blob=blob.name)
    with open(os.path.join(dataset_dir, blob.name), "wb") as my_blob:
        blob_data = blob_client.download_blob()
        blob_data.readinto(my_blob)
        print("\tBlob '{}' downloaded".format(blob.name))


# Create a dataset from the local file

dataset_name = "dataset-delays-flights"

dataset = Data(
    name=dataset_name,
    description="Dataset for delays flights prediction 2015",
    tags={"ama_owner": "romain.caret"},
    type=AssetTypes.URI_FOLDER,
    path=dataset_dir,
)

dataset = ml_client.data.create_or_update(dataset)

print(
    f"Dataset with name {dataset.name} is registered to workspace, the dataset version is {dataset.version}"
)

# Delete the local files

if os.path.isdir(dataset_dir):
    shutil.rmtree(dataset_dir)

airline_codes.csv
	Blob 'airline_codes.csv' downloaded
airlines.csv
	Blob 'airlines.csv' downloaded
airports.csv
	Blob 'airports.csv' downloaded
flights.csv
	Blob 'flights.csv' downloaded


Your file exceeds 100 MB. If you experience low upload speeds or latency, we recommend using the AzCopy tool for this file transfer. See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.
Uploading Datasets (592.49 MBs): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

Dataset with name dataset-delays-flights is registered to workspace, the dataset version is 1
